In [1]:
#Import required libaries
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

#KERAS
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.utils import np_utils

# SKLEARN
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
#Directory location for image data - USE OWN DIRECTORY HERE
DATADIR = '/Users/David/Desktop/CNN-image-classifier/training_data'
#Sub directories for different categories
CATEGORIES = ["10_1","10_3"]

In [3]:
training_data = []

In [4]:
#add all images to training data
def create_training_data():
    for category in CATEGORIES:  # 10_1, 10_2, 10_3

        path = os.path.join(DATADIR,category)  # create path to different stages
        class_num = CATEGORIES.index(category)  # get the classification  (0, 1 or 2)

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                training_data.append([img_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

In [5]:
create_training_data()

100%|██████████| 1008/1008 [00:00<00:00, 1075.80it/s]


In [6]:
#check all images have been added to training data
print(len(training_data))

2304


In [7]:
#shuffle training_data so that 10_1s and 10_3s are not together
import random

random.shuffle(training_data)

for sample in training_data[:10]:
    print(sample[1])

0
0
0
0
0
1
0
0
0
0


In [8]:
X = [] #Images
y = [] #Labels

# input image dimensions
img_rows, img_cols = 200, 200

In [9]:
for features, label in training_data:
    X.append(features)
    y.append(label)

In [10]:
X = np.array(X).reshape(-1, img_rows, img_cols, 1)

In [11]:
# Split X and y into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)


X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols,1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols,1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [12]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (1843, 200, 200, 1)
1843 train samples
461 test samples


In [14]:
#batch_size to train
batch_size = 32
# number of output classes
nb_classes = 2
# number of epochs to train
nb_epoch = 20

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

In [15]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [16]:
model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(img_rows, img_cols,1)))
convout1 = Activation('relu')
model.add(convout1)
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
convout2 = Activation('relu')
model.add(convout2)
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta')

Instructions for updating:
Colocations handled automatically by placer.


/Users/David/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(200, 200,..., padding="valid")`
  """
/Users/David/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  


In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
                verbose=1, validation_data=(X_test, Y_test))

Instructions for updating:
Use tf.cast instead.


/Users/David/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 1843 samples, validate on 461 samples
Epoch 1/20
1843/1843 [==============================] - 377s 205ms/step - loss: 0.8250 - accuracy: 0.6381 - val_loss: 0.5560 - val_accuracy: 0.7202
Epoch 2/20
1843/1843 [==============================] - 358s 194ms/step - loss: 0.5841 - accuracy: 0.7141 - val_loss: 0.5679 - val_accuracy: 0.7289
Epoch 3/20
1843/1843 [==============================] - 346s 188ms/step - loss: 0.5665 - accuracy: 0.7336 - val_loss: 0.5546 - val_accuracy: 0.7289
Epoch 4/20
1843/1843 [==============================] - 365s 198ms/step - loss: 0.5468 - accuracy: 0.7374 - val_loss: 0.5349 - val_accuracy: 0.7332
Epoch 5/20
1843/1843 [==============================] - 426s 231ms/step - loss: 0.5241 - accuracy: 0.7553 - val_loss: 0.5568 - val_accuracy: 0.7245
Epoch 6/20
1843/1843 [==============================] - 327s 177ms/step - loss: 0.4944 - accuracy: 0.7640 - val_loss: 0.5316 - val_accuracy: 0.7289
Epoch 7/20
1843/1843 [==============================] - 345s 187m

In [19]:
score = model.evaluate(X_test, Y_test, verbose=0)

In [20]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.6359502046072043
Test accuracy: 0.811279833316803
